### References:
[1] Maciejewski, Filip B., Jacob Biamonte, Stuart Hadfield, and Davide Venturelli. "[Improving quantum approximate optimization by noise-directed adaptive remapping.](https://arxiv.org/abs/2404.01412)" arXiv preprint arXiv:2404.01412 (2024).

[2] Maciejewski, Filip B., Bao G. Bach, Maxime Dupont, P. Aaron Lott, Bhuvanesh Sundar, David E. Bernal Neira, Ilya Safro, and Davide Venturelli. "[A multilevel approach for solving large-scale qubo problems with noisy hybrid quantum approximate optimization.](https://arxiv.org/abs/2408.07793)" In 2024 IEEE High Performance Extreme Computing Conference (HPEC), pp. 1-10. IEEE, 2024.

[3] Maciejewski, Filip B., Stuart Hadfield, Benjamin Hall, Mark Hodson, Maxime Dupont, Bram Evert, James Sud et al. "[Design and execution of quantum circuits using tens of superconducting qubits and thousands of gates for dense Ising optimization problems.](https://arxiv.org/abs/2308.12423)" Physical Review Applied 22, no. 4 (2024): 044074.

[4] Tam, Wai-Hong, Hiromichi Matsuyama, Ryo Sakai, and Yu Yamashiro. "[Enhancing NDAR with Delay-Gate-Induced Amplitude Damping.]"(https://arxiv.org/abs/2504.12628) arXiv preprint arXiv:2504.12628 (2025).

[5] Lykov, Danylo, Ruslan Shaydulin, Yue Sun, Yuri Alexeev, and Marco Pistoia. "[Fast simulation of high-depth qaoa circuits.](https://arxiv.org/abs/2309.04841)" In Proceedings of the SC'23 Workshops of The International Conference on High Performance Computing, Network, Storage, and Analysis, pp. 1443-1451. 2023.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os
import uuid
import numpy as np
from quapopt import AVAILABLE_SIMULATORS
from quapopt.additional_packages.ancillary_functions_usra import ancillary_functions as anf

os.makedirs('../temp', exist_ok=True)

### Generate a random Hamiltonian instance

In [29]:
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.data_analysis.data_handling import (COEFFICIENTS_TYPE,
                                                 COEFFICIENTS_DISTRIBUTION,
                                                 CoefficientsDistributionSpecifier,
                                                 HAMILTONIAN_MODELS)
from quapopt.data_analysis.data_handling import LoggingLevel

#more qubits so we observe some effects
number_of_qubits = 15
seed_cost_hamiltonian = 42

coefficients_type = COEFFICIENTS_TYPE.DISCRETE
coefficients_distribution = COEFFICIENTS_DISTRIBUTION.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

# We generate a Hamiltonian instance. In this case it's a random Sherrington-Kirkpatrick Hamiltonian
hamiltonian_model = HAMILTONIAN_MODELS.SherringtonKirkpatrick
localities = (1,2)
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True)

print("Class description (cost):", cost_hamiltonian.hamiltonian_class_description)
print("Instance description (cost):", cost_hamiltonian.hamiltonian_instance_description)

# if we wish, we can solve the Hamiltonian
if cost_hamiltonian.lowest_energy is None:
    cost_hamiltonian.solve_hamiltonian(both_directions=True)

ground_state_energy = cost_hamiltonian.ground_state_energy
highest_energy = cost_hamiltonian.highest_energy

cost_hamiltonian.hamiltonian_class_specifier.get_description_string()



File not found!
FILE NOT FOUND!
Class description (cost): HMN=SK;LOC=(1, 2);CFD=CT~DIS_CDN~UNI_CDP~low~-1_high~1_step~1
Instance description (cost): NOQ=15;HII=42


'HMN=SK;LOC=(1, 2);CFD=CT~DIS_CDN~UNI_CDP~low~-1_high~1_step~1'

### Noise-Directed Adaptive Remapping

* We will now run NDAR (see Ref.~[1]) in a loop with the optimizer we used in the previous notebook.
* We will use stronger readout noise model and fewer samples, because we simulate only ~20-qubit system, so it's pretty easy to find ground state even from suboptimal points with noise.
* Note: if not using GPU, the simulation might be a bit slow.
* We will use small number of function calls and samples for the same reason.
* Again, this noise is not realistic, but it is sufficient to demonstrate the NDAR algorithm.

In [30]:
from quapopt.optimization.QAOA.implementation.QAOARunnerSampler import QAOARunnerSampler
from quapopt.circuits.noise.simulation.ClassicalMeasurementNoiseSampler import \
    ClassicalMeasurementNoiseSampler, MeasurementNoiseType
from quapopt.optimization.parameter_setting import OptimizerType
from quapopt.optimization.parameter_setting.variational.scipy_tools.ScipyOptimizerWrapped import ScipyOptimizerWrapped

#Number of objective function calls
number_of_function_calls = 15
#Number of measurements to estimate the expectation values
number_of_samples = 30

#We can specify the QAOA depth here.
qaoa_depth = 1

#we can specify the details of classical optimizer here.
classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[(-np.pi, np.pi)] * 2*int(qaoa_depth),
                                            optimizer_name='COBYLA',
                                            optimizer_kwargs=None,
                                            basinhopping=True,
                                            basinhopping_kwargs={'niter': 3},
                                            starting_point=[0.05] * 2*int(qaoa_depth)
                                            )

# Fully asymmetric noise -- equivalent to amplitude damping at the end of the circuit
# Set large value to simulate very strong noise. Otherwise, it's too easy to solve for small problems and we can't observe effects of feedback loop in simulation.
p_01 = 0.9
p_10 = None
print('p_01:', p_01, 'p_10:', p_10)

p_01: 0.9 p_10: None


In [31]:
from quapopt.meta_algorithms.NDAR.NDARRunnerQAOA import NDARRunnerQAOA
from quapopt.meta_algorithms.NDAR import (ConvergenceCriterionNames,
                                          ConvergenceCriterion)

seed_main = 0
numpy_rng_sampling = np.random.default_rng(seed=seed_main)
numpy_rng_boltzmann = np.random.default_rng(seed=seed_main)
numpy_rng_noise = np.random.default_rng(seed=seed_main)

#Set up the classical measurement noise sampler
CMNS = ClassicalMeasurementNoiseSampler(noise_type=MeasurementNoiseType.TP_1q_identical,
                                        noise_description={'p_01': p_01,
                                                           'p_10': p_10},
                                        rng=numpy_rng_noise)

#Here we specify the sampler class and its arguments
#It is abstract because we sometimes use it with different samplers (not QAOA)
qaoa_sampler_class = QAOARunnerSampler
qaoa_sampler_kwargs = {'numpy_rng_sampling': numpy_rng_sampling}

#Heuristic parameter for potentially re-gauging towards energy-increasing states.
#Value 0.0 means always gauge-transforming.
temperature = 0.0

#Here we specify the logger arguments
logger_kwargs_ndar = {'experiment_folders_hierarchy':
                          ['SimulationResults',
                           'TestingNDAR',
                           f'TestRuns_{datetime.datetime.today().strftime("%Y-%m-%d")}_{np.random.randint(1000)}'],
                      'experiment_set_name': f'NDARTests',
                      'experiment_set_id':f'T={temperature}',
                      'experiment_instance_id':f'p_01={p_01}',
                      'table_name_prefix': f'NDARRuns'}



logging_level = LoggingLevel.DETAILED

#Specify the convergence criterion for NDAR
#Here we use the maximum number of unsuccessful trials.
#If after 5 iterations the NDAR does not find a better solution, it stops.
max_iterations = 5
ndar_convergence_criterion = ConvergenceCriterion(
    convergence_criterion_name=ConvergenceCriterionNames.MaxUnsuccessfulTrials,
    convergence_value=max_iterations)

input_hamiltonian_representations = [cost_hamiltonian.copy()]

# Create the NDAR runner instance.
ndar_runner = NDARRunnerQAOA(input_hamiltonian_representations=input_hamiltonian_representations,
                             convergence_criterion=ndar_convergence_criterion,
                             sampler_class=qaoa_sampler_class,
                             attractor_model=None,
                             logging_level=logging_level,
                             logger_kwargs=logger_kwargs_ndar
                             )
print("GROUNDS STATE ENERGY:", ground_state_energy)


#Note: For the sake of reducing simulation complexity when known solutions are known, the runner will actually break when reaching ground state.
best_result_ndar, optimization_history_ndar = ndar_runner.run_NDAR(
    #QAOA kwargs
    qaoa_depth=qaoa_depth,
    number_of_function_calls=number_of_function_calls,
    classical_optimizer=classical_optimizer,
    number_of_samples_per_function_call=number_of_samples,
    measurement_noise=CMNS,
    numpy_rng_sampling=numpy_rng_sampling,
    ##### NDAR KWARGS
    numpy_rng_boltzmann=numpy_rng_boltzmann,
    #Generate optimizer seed for each optimization in a loop (argument is iteration index)
    step_seed_generator=lambda x: x,
    show_progress_bar_ndar=True,
    temperature_NDAR=temperature,
    show_progress_bars_optimization=True
)



No existing metadata found for the specified experiment set. 
GROUNDS STATE ENERGY: -42.0


  0%|          | 0/1000 [00:00<?, ?it/s]

COBYLA: 20it [00:03,  5.15it/s]                        5.12it/s]


Iteration:  0
Best energy so far:  -26.0 (AR: 0.7949)


COBYLA: 20it [00:03,  5.09it/s]                        5.22it/s]


Iteration:  1
Best energy so far:  -36.0 (AR: 0.9231)


COBYLA: 20it [00:03,  5.18it/s]                        5.32it/s]


Iteration:  2
Best energy so far:  -38.0 (AR: 0.9487)


COBYLA: 20it [00:03,  5.27it/s]                        5.27it/s]


Iteration:  3
Best energy so far:  -42.0 (AR: 1.0)
FOUND GROUND STATE! breaking

Finished after  4 iterations.
Final best energy: np.float64(-42.0)
Optimization time: 15.515396780996525


In [32]:

from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner

#Here we can check if noiseless QAOA with the same number of samples and function calls would find the ground state
qaoa_runner_sampler = qaoa_sampler_class(hamiltonian_representations_cost=[cost_hamiltonian.copy()],
                                         hamiltonian_representations_phase=None,
                                         **qaoa_sampler_kwargs)
if 'cuda' in AVAILABLE_SIMULATORS:
    qaoa_runner_sampler.initialize_backend_qokit(qokit_backend='gpu')
else:
    qaoa_runner_sampler.initialize_backend_qiskit(qaoa_depth=qaoa_depth)

qaoa_optimizer = QAOAOptimizationRunner(qaoa_runner=qaoa_runner_sampler)

best_n_results_noiseless_qaoa, optimization_result_noiseless_qaoa = qaoa_optimizer.run_optimization(qaoa_depth=qaoa_depth,
                                                                      number_of_function_calls=number_of_function_calls * len(
                                                                          optimization_history_ndar),
                                                                      classical_optimizer=classical_optimizer,
                                                                      optimizer_seed=1,
                                                                      number_of_samples=number_of_samples,
                                                                      numpy_rng_sampling=numpy_rng_sampling,
                                                                      show_progress_bar=True,
                                                                      measurement_noise=None,
                                                                      )
anf.cool_print("BEST ENERGY OF NOISELESS QAOA:", best_n_results_noiseless_qaoa[0][0],'blue')
anf.cool_print("BEST ENERGY OF NDAR:", best_result_ndar[0],'red')
anf.cool_print("NDAR better than noiseless QAOA:", bool(best_result_ndar[0] < best_n_results_noiseless_qaoa[0][0]),'green')


STARTED SHARED QISKIT IBM SESSION (instance 0)


COBYLA: 80it [00:01, 43.37it/s]                        6.82it/s]

BEST ENERGY OF NOISELESS QAOA: np.float64(-42.0)
BEST ENERGY OF NDAR: np.float64(-42.0)
NDAR better than noiseless QAOA: False


### Reading data

* In a moment, we will want to visualize data.
* We could take the data directly from the output of optimization, but here we will read it from database to test the logging:
* To this aim, we use a helper function imported from NDAR submodule

In [33]:
from quapopt.meta_algorithms.NDAR.NDARRunnerQAOA import gather_NDAR_qaoa_results
from quapopt.data_analysis.data_handling import (STANDARD_NAMES_DATA_TYPES as SNDT,
                                                 STANDARD_NAMES_VARIABLES as SNV, )

what_data = SNDT.NDAROverview
ndar_data = gather_NDAR_qaoa_results(input_hamiltonian_representations=input_hamiltonian_representations,
                                     sampler_class=qaoa_sampler_class,
                                     logger_kwargs=logger_kwargs_ndar,
                                     data_type=what_data)

ndar_data

,NDARIteration,Bitflip,AttractorModel,ConvergenceCriterion,TrialIndex,HamiltonianRepresentationIndex,Angles-0,Angles-1,EnergyMean,EnergyBest,BitstringBest,ExperimentInstanceID
0,0,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,9,0,0.139838,0.336606,2.000000,-26.0,"(np.int64(0), np.int64(0), np.int64(0), np.int...",p_01=0.9
0,1,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,9,0,-0.038341,0.320036,-21.000000,-36.0,"(np.int64(0), np.int64(0), np.int64(0), np.int...",p_01=0.9
0,2,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,0,0,0.050000,0.050000,-33.266667,-38.0,"(np.int64(0), np.int64(0), np.int64(0), np.int...",p_01=0.9
0,3,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,6,0,0.300798,0.258148,-33.666667,-42.0,"(np.int64(0), np.int64(0), np.int64(0), np.int...",p_01=0.9


### Visualization

* Let's take a look at a simple visualization of the NDAR optimization.

In [34]:
import plotly
import plotly.graph_objects as go

plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

In [35]:
xs = ndar_data[SNV.NDARIteration.id_long].tolist()
ys = ndar_data[SNV.EnergyBest.id_long].tolist()
#convert to approximation ratios
ys = [(highest_energy - Ei) / (highest_energy - ground_state_energy) for Ei in ys]

ndar_figure = go.Figure()
ndar_figure.add_trace(go.Scatter(x=xs, y=ys, mode='lines+markers', name=f'NDAR p(0|1) = {p_01}'))

ndar_figure.update_layout(
    title=f"NDAR (T = {temperature}) Performance for {number_of_qubits} qubits on {cost_hamiltonian.hamiltonian_class_specifier.get_description_string()}",
    xaxis_title="NDAR Iteration",
    yaxis_title="Approximation Ratio",
    template="plotly",
    font=dict(size=14),
    hovermode="x unified"
)
ndar_figure.update_yaxes(range=[0.65, 1.03])
#update xticks to only NDAR iterations:
ndar_figure.update_xaxes(tickvals=xs)

plotly.offline.plot(ndar_figure,
                    filename=f'../temp/NDAR_performance.html')
ndar_figure.show()